# Hugging Face Accelerate

`Accelerate` es una biblioteca de Hugging Face que permite ejecutar el mismo código PyTorch en cualquier configuración distribuida añadiendo sólo cuatro líneas de código. En resumen, entrenamiento e inferencia a escala de forma sencilla, eficiente y adaptable.

## Instalación

Para instalar `accelerate` con `pip` simplemente ejecuta:

``` bash
pip install accelerate
```

Y con `conda`:

``` bash
conda install -c conda-forge accelerate
```

``` python
pip install accelerate deepspeed
```

## Configuración

En cada entorno en el que se intale `accelerate` lo primero que se tiene que hacer es configurarlo, para ello ejecutamos en una terminal:

``` bash
accelerate config
```

In [1]:
!accelerate config

--------------------------------------------------------------------------------
In which compute environment are you running?
This machine
--------------------------------------------------------------------------------
multi-GPU
How many different machines will you use (use more than 1 for multi-node training)? [1]: 1
Should distributed operations be checked while running for errors? This can avoid timeout issues but will be slower. [yes/NO]: no
Do you wish to optimize your script with torch dynamo?[yes/NO]:no
Do you want to use DeepSpeed? [yes/NO]: no
Do you want to use FullyShardedDataParallel? [yes/NO]: no
Do you want to use Megatron-LM ? [yes/NO]: no
How many GPU(s) should be used for distributed training? [1]:2
What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:0,1
--------------------------------------------------------------------------------
Do you wish to use FP16 or BF16 (mixed precision)?
fp16
accelerate configuration saved at 

En mi caso las respuestas han sido
 * In which compute environment are you running?
    - [x] "This machine"
    - [_] "AWS (Amazon SageMaker)"
 > Quiero configurarlo en mi ordenador

 * Which type of machine are you using?
    - [_] multi-CPU
    - [_] multi-XPU
    - [x] multi-GPU
    - [_] multi-NPU
    - [_] TPU
 > Como tengo 2 GPUs y quiero ejecutar códigos distribuidos en ellas elijo `multi-GPU`
 
 * How many different machines will you use (use more than 1 for multi-node training)? [1]:
    - 1
 > Elijo `1` porque solo voy a ejecutar en mi ordenador

 * Should distributed operations be checked while running for errors? This can avoid timeout issues but will be slower. [yes/NO]:
    - no
 > Con esta opción, se puede elegir que `accelerate` chequee errores en la ejecución, pero haría que vaya más lento, así que elijo `no`, y en caso de que haya errores lo cambio a `yes`
 
 * Do you wish to optimize your script with torch dynamo?[yes/NO]:
    - no
 > De momento elijo `no`, pero más adelante veremos qué es esto

 * Do you want to use FullyShardedDataParallel? [yes/NO]:
    - no
 > De momento elijo `no`, pero más adelante veremos qué es esto
 
 * Do you want to use Megatron-LM ? [yes/NO]:
    - no
 > De momento elijo `no`, pero más adelante veremos qué es esto
 
 * How many GPU(s) should be used for distributed training? [1]:
    - 2
 > Elijo `2` porque tengo 2 GPUs

 * What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:
    - 0,1
 > Elijo `0,1` porque quiero usar las dos GPUs

 * Do you wish to use FP16 or BF16 (mixed precision)?
    - [_] no
    - [x] fp16
    - [_] bf16
    - [_] fp8
 > De momento elijo `fp16`, pero más adelante veremos qué es esto

La configuración se guardará en `~/.cache/huggingface/accelerate/default_config.yaml` y se puede modificar en cualquier momento. Vamos a ver qué hay dentro

In [ ]:
!cat ~/.cache/huggingface/accelerate/default_config.yaml

compute_environment: LOCAL_MACHINE
debug: false
distributed_type: MULTI_GPU
downcast_bf16: 'no'
gpu_ids: 0,1
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false


Otra forma de ver la configuración que tenemos es ejecutando en una terminal:

``` bash
accelerate env
```

In [ ]:
!accelerate env


Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.28.0
- Platform: Linux-5.15.0-105-generic-x86_64-with-glibc2.31
- Python version: 3.11.8
- Numpy version: 1.26.4
- PyTorch version (GPU?): 2.2.1+cu121 (True)
- PyTorch XPU available: False
- PyTorch NPU available: False
- System RAM: 31.24 GB
- GPU type: NVIDIA GeForce RTX 3090
- `Accelerate` default config:
	- compute_environment: LOCAL_MACHINE
	- distributed_type: MULTI_GPU
	- mixed_precision: fp16
	- use_cpu: False
	- debug: False
	- num_processes: 2
	- machine_rank: 0
	- num_machines: 1
	- gpu_ids: 0,1
	- rdzv_backend: static
	- same_network: True
	- main_training_function: main
	- downcast_bf16: no
	- tpu_use_cluster: False
	- tpu_use_sudo: False
	- tpu_env: []


Una vez hemos configurado `accelerate` podemos probar si lo hemos hecho bien ejecutando en una terminal:

``` bash
accelerate test
```

In [ ]:
!accelerate test


Running:  accelerate-launch ~/miniconda3/envs/nlp/lib/python3.11/site-packages/accelerate/test_utils/scripts/test_script.py
stdout: **Initialization**
stdout: Testing, testing. 1, 2, 3.
stdout: Distributed environment: DistributedType.MULTI_GPU  Backend: nccl
stdout: Num processes: 2
stdout: Process index: 0
stdout: Local process index: 0
stdout: Device: cuda:0
stdout: 
stdout: Mixed precision type: fp16
stdout: 
stdout: Distributed environment: DistributedType.MULTI_GPU  Backend: nccl
stdout: Num processes: 2
stdout: Process index: 1
stdout: Local process index: 1
stdout: Device: cuda:1
stdout: 
stdout: Mixed precision type: fp16
stdout: 
stdout: 
stdout: **Test process execution**
stdout: 
stdout: **Test split between processes as a list**
stdout: 
stdout: **Test split between processes as a dict**
stdout: 
stdout: **Test split between processes as a tensor**
stdout: 
stdout: **Test random number generator synchronization**
stdout: All rng are properly synched.
stdout: 
stdout: **Da

Vemos que termina diciendo `Test is a success! You are ready for your distributed training!` por lo que todo está correcto.

## Optimización del entrenamiento

### Código base

Vamos a hacer primero un código de entrenamiento base y luego lo optimizaremos  para ver cómo se hace y cómo mejora

Primero vamos a buscar un dataset, en mi caso voy a usar el dataset [tweet_eval](https://huggingface.co/datasets/tweet_eval)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emoji")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [ ]:
dataset["train"].info

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['❤', '😍', '😂', '💕', '🔥', '😊', '😎', '✨', '💙', '😘', '📷', '🇺🇸', '☀', '💜', '😉', '💯', '😁', '🎄', '📸', '😜'], id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='tweet_eval', config_name='emoji', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=3808792, num_examples=45000, shard_lengths=None, dataset_name='tweet_eval'), 'test': SplitInfo(name='test', num_bytes=4262151, num_examples=50000, shard_lengths=None, dataset_name='tweet_eval'), 'validation': SplitInfo(name='validation', num_bytes=396704, num_examples=5000, shard_lengths=None, dataset_name='tweet_eval')}, download_checksums={'hf://datasets/tweet_eval@b3a375baf0f409c77e6bc7aa35102b7b3534f8be/emoji/train-00000-of-00001.parquet': {'num_bytes': 2609973, 'checksum': None}, 'hf://datasets/tweet_eval@b3a375baf0f409c77e6bc7aa35

In [ ]:
print(dataset["train"].info.features["label"].names)

['❤', '😍', '😂', '💕', '🔥', '😊', '😎', '✨', '💙', '😘', '📷', '🇺🇸', '☀', '💜', '😉', '💯', '😁', '🎄', '📸', '😜']


In [ ]:
num_classes = len(dataset["train"].info.features["label"].names)
num_classes

20

Vemos que el dataset tiene 20 clases

Vamos a ver la secuencia máxima de cada split

In [ ]:
max_len_train = 0
max_len_val = 0
max_len_test = 0

split = "train"
for i in range(len(dataset[split])):
    len_i = len(dataset[split][i]["text"])
    if len_i > max_len_train:
        max_len_train = len_i
split = "validation"
for i in range(len(dataset[split])):
    len_i = len(dataset[split][i]["text"])
    if len_i > max_len_val:
        max_len_val = len_i
split = "test"
for i in range(len(dataset[split])):
    len_i = len(dataset[split][i]["text"])
    if len_i > max_len_test:
        max_len_test = len_i

max_len_train, max_len_val, max_len_test

(142, 139, 167)

Así que definimos la secuencia máximo en general como 130 para la tokeniazción

In [ ]:
max_len = 130

A nosotros nos interesa el dataset tokenizado, no con las secuencias en crudo, así que creamos un tokenizador

In [ ]:
from transformers import AutoTokenizer

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

Creamos una función de tokenización

In [ ]:
def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")

Y ahora tokenizamos el dataset

In [ ]:
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Como vemos ahora tenemos los tokens (`input_ids`) y las máscaras de atención (`attention_mask`), pero vamos a ver qué tipo de datos tenemos

In [ ]:
type(tokenized_dataset["train"][0]["input_ids"]), type(tokenized_dataset["train"][0]["attention_mask"]), type(tokenized_dataset["train"][0]["label"])

(list, list, int)

Así que vamos a cargarnos la feature `text` y convertir a tensores el resto

In [ ]:
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
type(tokenized_dataset["train"][0]["label"]), type(tokenized_dataset["train"][0]["input_ids"]), type(tokenized_dataset["train"][0]["attention_mask"])

(torch.Tensor, torch.Tensor, torch.Tensor)

Creamos un dataloader

In [ ]:
import torch
from torch.utils.data import DataLoader
BS = 64

dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

Cargamos el modelo

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)

Vamos a ver cómo es el modelo

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

Vamos a ver su última capa

In [ ]:
model.classifier.out_proj

Linear(in_features=768, out_features=2, bias=True)

In [ ]:
model.classifier.out_proj.in_features, model.classifier.out_proj.out_features

(768, 2)

Hemos visto que nuestro dataset tiene 20 clases, así que tenemos que modificar la última capa

In [ ]:
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)
model.classifier.out_proj

Linear(in_features=768, out_features=20, bias=True)

Ahora sí

Ahora creamos una función de loss

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

Un optimizador

In [ ]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=5e-4)

Y por último una métrica

In [ ]:
import evaluate

metric = evaluate.load("accuracy")

Vamos a ver con una muestra

In [ ]:
sample = next(iter(dataloader["train"]))

In [ ]:
sample["input_ids"].shape, sample["attention_mask"].shape

(torch.Size([64, 130]), torch.Size([64, 130]))

In [ ]:
model.to("cuda")
ouputs = model(input_ids=sample["input_ids"].to("cuda"), attention_mask=sample["attention_mask"].to("cuda"))
ouputs.logits.shape

torch.Size([64, 20])

In [ ]:
predictions = torch.argmax(ouputs.logits, axis=-1)
predictions.shape

torch.Size([64])

In [ ]:
loss = loss_function(ouputs.logits, sample["label"].to("cuda"))
loss.item()

2.9990389347076416

In [ ]:
accuracy = metric.compute(predictions=predictions, references=sample["label"])["accuracy"]
accuracy

0.015625

Ya podemos crear un pequeño bucle de entrenamiento

In [ ]:
from fastprogress.fastprogress import master_bar, progress_bar

epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

master_progress_bar = master_bar(range(epochs))
for i in master_progress_bar:
    model.train()
    progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        master_progress_bar.child.comment = f'loss: {loss}'

        loss.backward()
        optimizer.step()

    model.eval()
    progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
    master_progress_bar.main_bar.comment = f"Validation accuracy: {accuracy['accuracy']}\n"

### Código base en una única celda

Ya tenemos un código base, este es el código base de un entrenamiento en pytorch, vamos a escribirlo todo en una única celda para ir cambiándola

In [ ]:
%%time

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from fastprogress.fastprogress import master_bar, progress_bar

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 64
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

master_progress_bar = master_bar(range(EPOCHS))
for i in master_progress_bar:
    model.train()
    progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        master_progress_bar.child.comment = f'loss: {loss}'

        loss.backward()
        optimizer.step()

    model.eval()
    progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
    master_progress_bar.main_bar.comment = f"Validation accuracy: {accuracy['accuracy']}\n"
print(f"Accuracy = {accuracy['accuracy']}")

Accuracy = 0.2112
CPU times: user 3min 27s, sys: 670 ms, total: 3min 27s
Wall time: 3min 33s


### Código con accelerate

Ahora reemplazamos algunas cosas

En primer lugar importamos `Accelerator` y lo inicializamos

``` python
from accelerate import Accelerator
accelerator = Accelerator()
```

Ya no hacemos el típico

``` python 


Ahora reemplazamos algunas cosas

 * En primer lugar importamos `Accelerator` y lo inicializamos

``` python
from accelerate import Accelerator
accelerator = Accelerator()
```

 * Ya no hacemos el típico

``` python 
torch.device("cuda" if torch.cuda.is_available() else "cpu")
```

 * Sino que dejamos que sea `acelerate` el que elija el dispositivo mediante

``` python
device = accelerator.device
```

 * Pasamos los elementos relevantes para el entrenamiento por el método `prepare` y ya no hacemos `model.to(device)`

``` python
model, optimizer, dataloader["train"], dataloader["validation"] = preprare(model, optimizer, dataloader["train"], dataloader["validation"])
```

 * Ya no mandamos los datos y el modelo a la GPU con `.to(device)` ya que `accelerate` se ha encargado de ello con el método `prepare`

 * En vez de hacer el backpropagation con `loss.backward()` dejamos que lo haga `accelerate` con
 
``` python
accelerator.backward(loss)
```

 * A la hora de calcular la métrica en el bucle de validación, necesitamos recopilar los valores de todos los puntos, en caso de estar haciendo un entrenamiento distribuido, para ello hacemos

``` python
predictions = accelerator.gather_for_metrics(predictions)
```

In [1]:
%%time

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from fastprogress.fastprogress import master_bar, progress_bar

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 64
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

master_progress_bar = master_bar(range(EPOCHS))
for i in master_progress_bar:
    model.train()
    progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        master_progress_bar.child.comment = f'loss: {loss}'

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
    master_progress_bar.main_bar.comment = f"Validation accuracy: {accuracy['accuracy']}\n"

print(f"Accuracy = {accuracy['accuracy']}")

Accuracy = 0.2112
CPU times: user 3min 31s, sys: 1.68 s, total: 3min 33s
Wall time: 3min 37s


Vemos que antes tardó unos 3 minutos y medio y ahora también, es decir, parece que `accelerate` no ha hecho nada. Pero eso es por la configuración que hemos puesto cuando hemos hecho `accelerate config`.

Si lo vuelves a mirar, lo único que hicimos fue decirle que tenemos un ordenador con dos GPUs y ya está, no hemos dicho que queremos hacer nada distribuido. Así que a partir de aquí iremos viendo cómo hacerlo.